Based on https://learn.deeplearning.ai/chatgpt-prompt-eng/

## Setup

In [1]:
import openai

In [16]:
# Set API Key

openai.api_key="sk-..."
# or 
# !export OPENAI_API_KEY='sk-...'  - this did not work

In [14]:
# Code to run when using env variable for api key
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
# Helper function for prompting
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, max_tokens=200):
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        # max_tokens=max_tokens,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


## Prompting Guidelines
Provide Clear and Specific instructions
#### Tactic 1: Use delimiters
- Triple Quotes: """
- Triple backticks: ```
- Triple dashes: ---
- Angle brackets: <>
- XML tags: \<tag>\</tag> 

In [19]:
text = f"""
Generating random paragraphs can be an excellent way \
for writers to get their creative flow going at the beginning \
of the day. The writer has no idea what topic the random paragraph \
will be about when it appears. This forces the writer to use creativity \
to complete one of three common writing challenges. The writer can use the \
paragraph as the first one of a short story and build upon it. A second option \
is to use the random paragraph somewhere in a short story they create. The \
third option is to have the random paragraph be the ending paragraph in a \
short story. No matter which of these challenges is undertaken, the writer \
is forced to use creativity to incorporate the paragraph into their writing.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Tactic 2: Ask for a structured output
JSON, HTML

In [ ]:
prompt = f"""
Generate a list of three bollywood movies along \ 
with their director and year of release. 
Provide them in JSON format with the following keys: 
Movie Name, Year of Release, Director.
"""
response = get_completion(prompt)
print(response)

#### Tactic 3: Ask the model to check whether conditions are satisfied

In [ ]:
prompt = f"""
x=1. y=2. x+y=3
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains mathematical equations, \ 
re-write the equation as a list of steps in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain contains a mathematical equation, \ 
then simply write \"No equation provided.\"

\"\"\"{prompt}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Prompt:")
print(response)

#### Tactic 4 - Few-Shot Prompting: Provide examples of completing tasks. Then ask model to perform the task.

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<question>: Sholay.

<answer>: The movie "Sholay" was released in the 70s and started Amitabh \
and Dharmendra in the lead roles.

<question>: Karz.
"""
response = get_completion(prompt)
print(response)

#### Tactic 5: Specify the steps required to complete a task

In [ ]:
prompt = f"""
Perform the following actions: 
1 - Write a parable.
2 - Summarize it in a single line in Hindi.
3 - State the moral of the parable.
4 - Output a json object that contains the following \
keys: title, moral.

Separate your answers with line breaks.
"""
response = get_completion(prompt)
print("Completion for prompt:")
print(response)

#### Tactic 6: Ask for response in a specified format

In [ ]:
# Example 1 - Simple JSON
prompt = f"""
Your task is to perform the following actions: 
1 - Write a parable.
2 - Summarize it in a single line in Bangla.
3 - State the moral of the parable.
4 - Output a json object that contains the following \
keys: title, moral.

Use the following format:
Parable: <text to summarize>
Summary: <summary>
Moral: <moral of story>
Output JSON: <json with title and moral>
"""
response = get_completion(prompt)
print("\nCompletion for prompt:")
print(response)

In [ ]:
# Example 2 - HTML Table with datatype
prompt = f"""
Your task is to perform the following actions: 
1. Prepare a list of 10 bollywood movies
2. Find out year each of the movies was released
3. Find out if the movie was a hit or a flop at the box office.

Output the response as a table having three columns.
In the first column include the name of the movie.  Use "Name" as the column title.
In the second column include the year of release (Use "Year" as the column title ) \
and in the third column, include the box office result. Use "Hit or Flop" as the \
column title and format the value as a boolean.

Give the table the title 'Movie Stats'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Output in markdown format as well. 
"""
response = get_completion(prompt)
print("\nCompletion for prompt:")
print(response)

In [3]:
from IPython.display import display, HTML
display(HTML(response))

In [ ]:
# Example 3 - Markdown
prompt = f"""
Output the following in markdown format: ```{text}```
"""
response = get_completion(prompt)
print("\nCompletion for prompt:")
print(response)

#### Tactic 7: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if \
the student's solution is correct or not. Don't decide if the student's \
solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm producing a television show and need help
working out the cost associated to the payments for the actors. 
- Lead actor charges $100 / episode
- Lead actress charges $90 / episode
- I negotiated a contract for character actors that will cost \
me a flat $1K for the first 10 character actors for the entire show, and an additional \
$25 per additional character actor / episode
What is the total amount I need to spend on the actors \
as a function of the number of episodes. Assume that I end up using a total of 12 \
character actors per episode.
``` 
Student's solution:
```
Let x be the number of episodes.
Costs:
1. Lead actor charges: 100x
2. Lead actress charges: 90x
3. Character actor charges: 1,000 + 300x
Total cost: 100x + 90x + 1,000 + 300x = 490x + 1,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

#### Tactic 8: Specify length of output
- Use at most 50 words.
- Use at most 3 sentences.
- Give your answer as a single word, either "this" or "that".
- Make your response as short as possible.

#### Tactic 9: Reducing hallucinations (fabricated responses)
Ask model to find relevant information, then answer the question based on the relevant information and provide citations.

## Summarizing

In [ ]:
# Example 1
prompt = prompt = f"""
Your task is to generate a short summary of a product \
review to give feedback to the pricing deparmtment.  

Summarize the review below, delimited by triple 
backticks, in at most 2 sentences, and focusing on any aspects \
that are relevant to the price and perceived value.
```{text}```
"""
response = get_completion(prompt)
print(response)

In [ ]:
# Example 2 - Use 'extract' instead of summarize
prompt = f"""
Your task is to generate a short summary of a product \
review to give feedback to the pricing deparmtment. 

From the review below, delimited by triple quotes \
extract the information relevant to the price and perceived value. \
Limit to 2 sentences. 
```{text}```
"""

response = get_completion(prompt)
print(response)

## Inferring

#### Sentiments and Emotions

In [ ]:
# Example 1
prompt = f"""
What is the sentiment of the writer in the followin text, 
which is delimited with triple backticks?
Give your answer as a single word, either "positive" or "negative".
```{text}```
"""
response = get_completion(prompt)
print(response)

In [ ]:
# Example 2
prompt = f"""
Identify a list of emotions expressed by the writer of the \
following review . Include no more than three items in the list. \
Format your answer as a comma separated list in upper case. 
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Specific Pieces of Information

In [ ]:
prompt = f"""
Identify the following items from the text delimited with triple backticks: 
- Movie Name
- Year Of Release

Format your response as a JSON object with "Name" and "Year" as the keys. 
If the information isn't present, use "unknown" as the value.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### Topics

In [ ]:
# Example 1
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.
Make each item one or two words long. 
Format your response as a list of items separated by commas.
```{text}```
"""
response = get_completion(prompt)
print(response)

In [ ]:
# Example 2 - Zero Shot Algorithm
topic_list = ["topic1", "topic2", "topic3", "topic4", "topic5"]
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.
Format the response as a JSON  with the following keys: topic, included

List of topics: {", ".join(topic_list)}
```{text}```
"""
response = get_completion(prompt)
print(response)

## Transforming

#### Language Translation

In [ ]:
prompt = f"""
Translate the following text delimited by triple backticks to bengali. \
Provide both formal and informal translations. 
```Who are you```
"""
response = get_completion(prompt)
print(response)

#### Tone Transformation

In [ ]:
prompt = f"""
Translate the following from slang to a business letter: 
'Holy shit, This stuff don't work. Get me a new one.'
"""
response = get_completion(prompt)
print(response)

#### Format Conversion

In [ ]:
prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {json_text}
"""
response = get_completion(prompt)
print(response)

#### Grammer Check

In [ ]:
prompt = f"""Proofread and correct the following text \
and rewrite the corrected version. Make it more compelling. \
```{text}```"""
response = get_completion(prompt)
print(response)

## Email Response

In [ ]:
prompt = f"""
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, identify the sentiment of the email. \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
```{email_text}```
"""
response = get_completion(prompt)
print(response)